Note that I am running this on google colab

# Introduction
Date: 5/10/2021.\
Name: Huynh Phuong Nhu.\
Student ID: 19520208.\


What to do?

Bước 1: Load bài nhạc

Bước 2: Nghe thử bài nhạc gốc

Bước 3: Reshape từ array sang matrix kích thước HxW (H=512)

Bước 3: Dùng PCA để giảm số chiều H -> h

Bước 4: Khôi phục lại âm thanh

Bước 5: Nghe lại đoạn âm thanh đã khôi phục

In [1]:
!pip install pydub

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from pydub import AudioSegment

song = AudioSegment.from_mp3("/content/drive/MyDrive/Data/Nekkoya-PRODUCE-48.mp3")
song.export("output.wav", format = "wav") 

<_io.BufferedRandom name='output.wav'>

In [4]:
import numpy as np
from sklearn.decomposition import PCA

In [13]:
from IPython.display import Audio
from scipy.io import wavfile
samplerate, tabulasa = wavfile.read('output.wav')

start = samplerate * 14 # 10 seconds in
end = start + samplerate * 10 # 5 second duration
Audio(data=tabulasa[start:end, 0], rate=samplerate)

In [8]:
def pca_reduce(signal, n_components, block_size=512):

    # First, zero-pad the signal so that it is divisible by the block_size
    samples = len(signal)
    hanging = block_size - np.mod(samples, block_size)
    padded = np.lib.pad(signal, (0, hanging), 'constant', constant_values=0)

    # Reshape the signal to have block_size dimensions
    reshaped = padded.reshape((len(padded) // block_size, block_size))

    # Second, do the actual PCA process
    pca = PCA(n_components=n_components)
    pca.fit(reshaped)

    transformed = pca.transform(reshaped)
    reconstructed = pca.inverse_transform(transformed).reshape((len(padded)))
    return pca, transformed, reconstructed

In [14]:
# Reduce it down to 64/1024 
# Take the right chanel
temp = tabulasa[:,1]
_, _, reconstructed = pca_reduce(temp, 64, 1024)
Audio(data=reconstructed[start:end], rate=samplerate)

In [15]:
# Take the left channel and reduce it to 64
# Sound much clearer compare to the above
temp = tabulasa[:,0]
_, _, reconstructed = pca_reduce(temp, 64, 512)
Audio(data=reconstructed[start:end], rate=samplerate)